# Training and Testing the different ML models for telco customer churn perdiction


Train and track multiple models with MLflow, such as:
- Logistic Regression
- Random Forest
- XGBoost
- LightGBM

Compare model performance using metrics such as AUC, F1-score, and Accuracy.


# Prepare data

In [6]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
# load cleanded dataset
data = pd.read_csv(os.path.join("..", "data", "processed", "cleaned_telco_customer_churn.csv"))
data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,29.85,29.85,0
1,0,0,0,34,1,1,1,0,1,0,0,0,1,0,1,56.95,1889.50,0
2,0,0,0,2,1,1,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,0,0,0,45,0,1,1,0,1,1,0,0,1,0,2,42.30,1840.75,0
4,0,0,0,2,1,2,0,0,0,0,0,0,0,1,0,70.70,151.65,1


In [5]:
target = data['Churn']
features = data.drop(columns=['Churn'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42, stratify=target
)

# Define Models and Params

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [27]:
models = {
    'logistic_regression': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('logreg', LogisticRegression())
        ]),
        'params': {
            'logreg__C': [0.01, 0.1, 1, 10, 100],
            'logreg__solver': ['liblinear', 'lbfgs']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30]
        }
    },
    'xgboost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7]
        }
    },
    'LightGBM': {
        'model': LGBMClassifier(),
        'params': {
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [50, 100, 200],
            'num_leaves': [31, 50, 100]
        }
    }
}

## Train different models and track it in mlflow

In [30]:
import mlflow
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [31]:

mlflow.set_experiment("Telco_Churn_Models")
mlflow.set_tracking_uri("http://localhost:5000")

2025/10/19 00:27:02 INFO mlflow.tracking.fluent: Experiment with name 'Telco_Churn_Models' does not exist. Creating a new experiment.


In [32]:
mlflow.set_experiment("Telco_Churn_Models")

for model_name, mp in models.items():
    print(f"Training {model_name}...")
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='f1', n_jobs=-1)
    clf.fit(X_train, y_train)

    best_model = clf.best_estimator_
    y_pred = best_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("best_params", clf.best_params_)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        if model_name == 'logistic_regression':
            mlflow.sklearn.log_model(best_model, artifact_path="model")
        elif model_name == 'random_forest':
            mlflow.sklearn.log_model(best_model, artifact_path="model")
        elif model_name == 'xgboost':
            mlflow.xgboost.log_model(best_model, artifact_path="model")
        elif model_name == 'LightGBM':
            mlflow.lightgbm.log_model(best_model, artifact_path="model")
        
    print(f"{model_name} - F1 Score: {f1}")

Starting model training and logging to MLflow...
Training logistic_regression...


Best parameters for logistic_regression: {'logreg__C': 0.01, 'logreg__solver': 'liblinear'}
logistic_regression - Accuracy: 0.7920511000709723, Precision: 0.6246153846153846, Recall: 0.5427807486631016, F1 Score: 0.580829756795422


2025/10/19 00:27:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 00:27:33 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/19 00:27:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run logistic_regression at: http://localhost:5000/#/experiments/878427477543592209/runs/2421c33e761243c1b0bf8c9a676130d0
🧪 View experiment at: http://localhost:5000/#/experiments/878427477543592209
logistic_regression - F1 Score: 0.580829756795422
Training random_forest...
Best parameters for random_forest: {'max_depth': 10, 'n_estimators': 100}
random_forest - Accuracy: 0.794889992902768, Precision: 0.6366559485530546, Recall: 0.5294117647058824, F1 Score: 0.5781021897810219


2025/10/19 00:27:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 00:28:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/19 00:28:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Best parameters for xgboost: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50}
xgboost - Accuracy: 0.7955997161107168, Precision: 0.6433333333333333, Recall: 0.516042780748663, F1 Score: 0.5727002967359051


2025/10/19 00:28:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/mnt/sda1/ML-Projects/telco-churn-mlops-pipeline/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1115: UserWarning: [00:28:13] WARNING: /workspace/src/c_api/c_api.cc:1570: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/10/19 00:28:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/19 00:28:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgboost at: http://localhost:5000/#/experiments/878427477543592209/runs/f27a34342f0141f599680b3a97496fdc
🧪 View experiment at: http://localhost:5000/#/experiments/878427477543592209
xgboost - F1 Score: 0.5727002967359051
Training LightGBM...
[LightGBM] [Info] Number of positive: 1196, number of negative: 3311
[LightGBM] [Info] Number of positive: 1196, number of negative: 3311
[LightGBM] [Info] Number of positive: 1196, number of negative: 3311
[LightGBM] [Info] Number of positive: 1196, number of negative: 3311
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005032 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006606 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.

[LightGBM] [Info] A

2025/10/19 00:29:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 00:29:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/19 00:29:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM at: http://localhost:5000/#/experiments/878427477543592209/runs/2515a1afbfc94130836890cfe476c843
🧪 View experiment at: http://localhost:5000/#/experiments/878427477543592209
LightGBM - F1 Score: 0.5855072463768116
